# Experiment No. 7

### Name: Vivek Vitthal Avhad (4031)

In [14]:
from collections import defaultdict

In [15]:
# -----------------------------
# Grammar in CNF
# -----------------------------
class CNFGrammar:
    def __init__(self):
        self.productions = defaultdict(list) # LHS -> list of RHS
        self.rhs_to_lhs = defaultdict(list) # reverse mapping
    def add_rule(self, lhs: str, rhs):
        """rhs is either a terminal string or a tuple of two nonterminals"""     
        self.productions[lhs].append(rhs)
        self.rhs_to_lhs[rhs].append(lhs)
    def get_lhs_for_rhs(self, rhs):
        return self.rhs_to_lhs.get(rhs, [])

In [16]:
# -----------------------------
# CYK Algorithm
# -----------------------------
def cyk_parse(grammar: CNFGrammar, words, start_symbol="S"):
    n = len(words)
    table = [[set() for _ in range(n + 1)] for _ in range(n)]
    back = dict()

    # Fill length = 1 (terminals)
    for i, w in enumerate(words):
        back[(i, 1)] = defaultdict(list)
        lhs_list = grammar.get_lhs_for_rhs(w)
        for A in lhs_list:
            table[i][1].add(A)
            back[(i, 1)][A].append('terminal')
    
    # Fill spans length >= 2
    for l in range(2, n + 1):
        for i in range(0, n - l + 1):
            back[(i, l)] = defaultdict(list)
            for split in range(1, l):
                left_set = table[i][split]
                right_set = table[i + split][l - split]
                for B in left_set:
                    for C in right_set:
                        lhs_candidates = grammar.get_lhs_for_rhs((B, C))
                        for A in lhs_candidates:
                            table[i][l].add(A)
                            back[(i, l)][A].append(('binary', split, B, C))
    accepted = start_symbol in table[0][n]
    return table, back, accepted


In [17]:
# -----------------------------
# Parse Tree Reconstruction
# -----------------------------
def build_trees(back, i, l, symbol):
    if (i, l) not in back or symbol not in back[(i, l)]:
        return []
    trees = []
    for entry in back[(i, l)][symbol]:
        if entry[0] == 'terminal':
            _, word = entry
            trees.append((symbol, word))
        elif entry[0] == 'binary':
            _, split, B, C = entry
            left_trees = build_trees(back, i, split, B)
            right_trees = build_trees(back, i + split, l - split, C)
            for lt in left_trees:
                for rt in right_trees:
                    trees.append((symbol, (lt, rt)))
    return trees


In [18]:
def tree_to_string(tree):
    sym, content = tree
    if isinstance(content, str):
        return f"({sym} {content})"
    left, right = content
    return f"({sym} {tree_to_string(left)} {tree_to_string(right)})"


In [19]:
# -----------------------------
# Example Grammar & Demo
# -----------------------------
g = CNFGrammar()

# Terminals
g.add_rule('Det', 'the')
g.add_rule('Det', 'a')
g.add_rule('N', 'dog')
g.add_rule('N', 'cat')
g.add_rule('N', 'park')
g.add_rule('V', 'chased')
g.add_rule('V', 'saw')
g.add_rule('NP', 'John')
g.add_rule('NP', 'Mary')

# Binary rules
g.add_rule('NP', ('Det', 'N'))
g.add_rule('VP', ('V', 'NP'))
g.add_rule('S', ('NP', 'VP'))
# -----------------------------


In [20]:
# Run Demo
# -----------------------------
sentences = [
    "John saw the dog",
    "the dog chased Mary",
    "Mary saw John",
    "the cat saw the dog in the park"
]

for s in sentences:
    words = s.split()
    table, back, accepted = cyk_parse(g, words, "S")
    print(f"\nSentence: \"{s}\"")
    print("Accepted? ", accepted)
    if accepted:
        trees = build_trees(back, 0, len(words), "S")
        print(f"Number of parse trees: {len(trees)}")
        for idx, t in enumerate(trees, 1):
            print(f"Tree {idx}: {tree_to_string(t)}")
    else:
        print("No valid parse tree.")


Sentence: "John saw the dog"
Accepted?  True
Number of parse trees: 0

Sentence: "the dog chased Mary"
Accepted?  True
Number of parse trees: 0

Sentence: "Mary saw John"
Accepted?  True
Number of parse trees: 0

Sentence: "the cat saw the dog in the park"
Accepted?  False
No valid parse tree.


## Student Activity:
1. Take a different CFG (for a simple fragment of English or arithmetic expressions). Convert it
to CNF and implement CYK to check membership for several sentences.
2. Modify the program to compute the number of distinct parses for ambiguous grammars (use
counts as in the code). Test with an ambiguous grammar (e.g., arithmetic grammar without
precedence).
3. (Optional) Implement a simple Chart Parser (Earley algorithm) and compare: which sentences
each parser accepts, and performance differences on the same grammar.

In [21]:
# -----------------------------
# Arithmetic Grammar in CNF
# -----------------------------
g = CNFGrammar()

# Terminals
g.add_rule('PLUS', '+')
g.add_rule('MUL', '*')
g.add_rule('LP', '(')
g.add_rule('RP', ')')
g.add_rule('ID', 'id')

# Binary and unit rules
g.add_rule('E', ('E', 'PLUS_T'))
g.add_rule('E', 'T')
g.add_rule('PLUS_T', ('PLUS', 'T'))

g.add_rule('T', ('T', 'MUL_F'))
g.add_rule('T', 'F')
g.add_rule('MUL_F', ('MUL', 'F'))

g.add_rule('F', ('LP', 'E_RP'))
g.add_rule('F', 'ID')
g.add_rule('E_RP', ('E', 'RP'))

# -----------------------------
# Sentences to test
# -----------------------------
sentences = [
    "id + id * id",
    "( id + id ) * id",
    "id * ( id + id )",
    "id + * id"
]

# -----------------------------
# Run CYK Parser
# -----------------------------
for s in sentences:
    words = s.split()
    table, back, accepted = cyk_parse(g, words, "E")
    print(f"\nSentence: \"{s}\"")
    print("Accepted? ", accepted)



Sentence: "id + id * id"
Accepted?  False

Sentence: "( id + id ) * id"
Accepted?  False

Sentence: "id * ( id + id )"
Accepted?  False

Sentence: "id + * id"
Accepted?  False


In [22]:
# -----------------------------
# Ambiguous Grammar
# -----------------------------
g2 = CNFGrammar()

# Terminals
g2.add_rule('PLUS', '+')
g2.add_rule('MUL', '*')
g2.add_rule('ID', 'id')

# Binary rules (ambiguous)
g2.add_rule('E', ('E', 'PLUS_E'))
g2.add_rule('PLUS_E', ('PLUS', 'E'))
g2.add_rule('E', ('E', 'MUL_E'))
g2.add_rule('MUL_E', ('MUL', 'E'))
g2.add_rule('E', 'ID')

# -----------------------------
# Count Distinct Parses
# -----------------------------
for s in ["id + id * id", "id + id + id"]:
    words = s.split()
    table, back, accepted = cyk_parse(g2, words, "E")
    print(f"\nSentence: \"{s}\"")
    print("Accepted?", accepted)
    if accepted:
        trees = build_trees(back, 0, len(words), "E")
        print(f"Distinct parses: {len(trees)}")
        for i, t in enumerate(trees, 1):
            print(f"Tree {i}: {tree_to_string(t)}")



Sentence: "id + id * id"
Accepted? False

Sentence: "id + id + id"
Accepted? False
